In [1]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../../src/')
from train import load_data
# from diffusion import DiffusionModel
# from evaluate import get_results
from omegaconf import OmegaConf
# from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch
from model2 import Autoencoder, Preprocessor, Discriminator
import magic
import torch
import pathlib
import copy

import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
from omegaconf import OmegaConf
import os
import glob
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from geodesic import jacobian, velocity, CondCurve, GeodesicBridgeOverfit
from plotly3d.plot import scatter, trajectories
import torch
from torch import nn
import pytorch_lightning as pl
from procrustes import Procrustes
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt
from tqdm import tqdm

wandb.login()
api = wandb.Api()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xingzhis. Use `wandb login --relogin` to force relogin


In [ ]:
data_path = "../../data/neurips_results/toy/visualize/"

In [2]:
# ours
entity = "xingzhis"
project = "dmae"
sweep_id = '72pm72bl'
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

In [4]:
names = df[(df['loss_epoch']!='NaN')][['data_name']].values.flatten()

In [6]:
pathlib.Path('results/ours').mkdir(parents=True, exist_ok=True)

In [42]:
for name in names:
    run_id = df[df['data_name'] == name]['run_id'].values[0]
    run = api.run(f"{entity}/{project}/{run_id}")
    folder_path = '../geodesic_on_datasets/wandb/'
    cfg = OmegaConf.create(run.config)
    folder_list = glob.glob(f"{folder_path}*{run.id}*")
    ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
    ckpt_path = ckpt_files[0]

    ofm = lambda x: x
    gbmodel = GeodesicBridgeOverfit.load_from_checkpoint(
        checkpoint_path=ckpt_path,
        func=ofm,
        # func = enc_func,
        # discriminator_func=disc_func_pen,
        # discriminator_func_for_grad=discriminator_func_for_grad,
        input_dim=3, 
        hidden_dim=64, 
        scale_factor=1, 
        symmetric=True, 
        num_layers=3, 
        n_tsteps=100, 
        lr=1e-3, 
        weight_decay=1e-3,
        discriminator_weight=0.,
        discriminator_func_for_grad_weight=0.,
        id_dim=1,
        id_emb_dim=1,
        density_weight=0.,
        length_weight=1.,
    )
    data = np.load(f"{data_path}/{name.split('_')[0]}.npz")
    x = torch.tensor(data['X'], dtype=torch.float32, device=gbmodel.device)
    xbatch = torch.tensor(data['start_points'], dtype=x.dtype, device=x.device)
    xendbatch = torch.tensor(data['end_points'], dtype=x.dtype, device=x.device)
    ids = torch.zeros((xbatch.size(0),1))
    x0 = xbatch
    x1 = xendbatch
    with torch.no_grad():
        cc_pts = gbmodel.cc(x0, x1, gbmodel.ts, ids)
    np.save(f"results/ours/{name.split('_')[0]}.npy", cc_pts.cpu().numpy())

In [43]:
# no density
entity = "xingzhis"
project = "dmae"
sweep_id = 'ro5674uj'
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

In [44]:
pathlib.Path('results/no_density').mkdir(parents=True, exist_ok=True)

In [45]:
for name in names:
    run_id = df[df['data_name'] == name]['run_id'].values[0]
    run = api.run(f"{entity}/{project}/{run_id}")
    folder_path = '../geodesic_on_datasets/wandb/'
    cfg = OmegaConf.create(run.config)
    folder_list = glob.glob(f"{folder_path}*{run.id}*")
    ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
    ckpt_path = ckpt_files[0]

    ofm = lambda x: x
    gbmodel = GeodesicBridgeOverfit.load_from_checkpoint(
        checkpoint_path=ckpt_path,
        func=ofm,
        # func = enc_func,
        # discriminator_func=disc_func_pen,
        # discriminator_func_for_grad=discriminator_func_for_grad,
        input_dim=3, 
        hidden_dim=64, 
        scale_factor=1, 
        symmetric=True, 
        num_layers=3, 
        n_tsteps=100, 
        lr=1e-3, 
        weight_decay=1e-3,
        discriminator_weight=0.,
        discriminator_func_for_grad_weight=0.,
        id_dim=1,
        id_emb_dim=1,
        density_weight=0.,
        length_weight=1.,
    )
    data = np.load(f"{data_path}/{name.split('_')[0]}.npz")
    x = torch.tensor(data['X'], dtype=torch.float32, device=gbmodel.device)
    xbatch = torch.tensor(data['start_points'], dtype=x.dtype, device=x.device)
    xendbatch = torch.tensor(data['end_points'], dtype=x.dtype, device=x.device)
    ids = torch.zeros((xbatch.size(0),1))
    x0 = xbatch
    x1 = xendbatch
    with torch.no_grad():
        cc_pts = gbmodel.cc(x0, x1, gbmodel.ts, ids)
    np.save(f"results/no_density/{name.split('_')[0]}.npy", cc_pts.cpu().numpy())